In [ ]:
import cv2
import cv2.dnn as dnn
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
import scipy

In [ ]:
def get_output_layers(net):    
    layer_names = net.getLayerNames()    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    # label = str(classes[class_id])
    # color = COLORS[class_id]
    # color = 'blue'
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), 1, 2)
    # cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
net = dnn.readNetFromDarknet('../darknet/darknet/cfg/yolov3.cfg', '../darknet/darknet/yolov3.weights')

In [ ]:
def get_bounding_boxes(image):
    scale = 1/255
    width = image.shape[1]
    height = image.shape[0]
    blob = dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))
    outs = net.forward()

    conf_threshold = 0.9
    nms_threshold = 0.45

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
            # if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = (center_x - w/2.0)
                y = (center_y - h/2.0)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    bboxes = []

    test = image
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = round(box[0])
        y = round(box[1])
        w = box[2]
        h = box[3]

        # draw_bounding_box(test, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        bboxes.append(image[y:y+h, x:w+x])

    # plt.imshow(test)

    # fig, axes = plt.subplots(1, len(bboxes))

    # if len(bboxes) == 1:
    #    axes.imshow(bboxes[0])
    #else:
    #    for i in range(len(bboxes)):
    #        axes[i].imshow(bboxes[i])

    if len(bboxes) == 1:
        return bboxes[0]
    return bboxes[1]

In [ ]:
class YoloBoundingBox:
    def __init__(self, x, y, w, h, width, height):
        center_x = int(x * width)
        center_y = int(y * height)
        self.top_left = [int(center_x / 2.0), int(center_y / 2.0)]
        self.bottom_right = [self.top_left[0] + height, self.top_left[1] + width]

In [ ]:
image  = cv2.imread('images/dog.jpg')
scale = 1/255
width = image.shape[1]
height = image.shape[0]
blob = dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))

conf_threshold = 0.9
nms_threshold = 0.45

class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        class_ids.append(class_id)
        confidences.append(float(confidence))
        boxes.append(detection[:4])

indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
bboxes = [YoloBoundingBox(*boxes[i[0]], width, height) for i in indices]

"""
bboxes = []
test = image
for i in indices:
    i = i[0]
    
    x = int(box[0])
    y = int(box[1])
    w = int(box[2])
    h = int(box[3])
    print(x,y,w,h)
    bboxes.append(image[y:h, x:w])
"""

In [ ]:
box = bboxes[1]

In [ ]:
plt.imshow(cv2.rectangle(image, tuple(box.top_left), tuple(box.bottom_right), (0,255,0))

In [ ]:
plt.imshow(image)

In [ ]:
alexnet = dnn.readNetFromCaffe('models/', 'bvlc_alexnet.caffemodel')

In [ ]:
alexnet = dnn.readNetFromTensorflow('inception/tensorflow_inception_graph.pb')

In [ ]:
def extract_feature(image, layer_name):
    alex_test_blob = dnn.blobFromImage(image, 1/255, (224,224), (0,0,0), swapRB=True, crop=False)
    alexnet.setInput(alex_test_blob)
    test_outs = alexnet.forward(layer_name)
    # print(test_outs.shape)
    # return test_outs.reshape(np.prod(test_outs.shape))
    return test_outs

In [ ]:
test_outs

In [ ]:
def cos_sim(a, b):
    # a = preprocessing.normalize(a.reshape(1,-1))
    # b = preprocessing.normalize(b.reshape(1,-1))
    # a = a.reshape(a.shape[1])
    # b = b.reshape(b.shape[1])
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    # return np.linalg.norm(a-b)

In [ ]:
def similarity(nda, ndb):
    nda = nda[0]
    ndb = ndb[0]
    
    t_nda = np.mean(nda, axis=0,keepdims=True)
    t_ndb = np.mean(ndb, axis=0, keepdims=True)
    
    t_nda = t_nda.reshape(np.prod(t_nda.shape))
    t_ndb = t_ndb.reshape(np.prod(t_ndb.shape))
    
    return cos_sim(t_nda, t_ndb)

In [ ]:
def logm(a):
    a = a[0]
    vec = []
    for i in range(a.shape[0]):
        vec.extend(scipy.linalg.logm(a[i], disp=False))
    
    return np.array(vec)

In [ ]:
def similarity(nda, ndb):
    loga = logm(nda)
    logb = logm(ndb)
    
    return cos_sim(loga, logb)
    

In [ ]:
from scipy.spatial import distance

In [ ]:
def manhattan_dist(a, b):
    return distance.cityblock(a,b)

In [ ]:
test_blob = dnn.blobFromImage(image, 1/255, (224,224), (0,0,0), swapRB=True, crop=False)
alexnet.setInput(test_blob)

test_outs = alexnet.forward('pool5')
# test_outs = test_outs.reshape((1, test_outs.shape[3], test_outs.shape[1], test_outs.shape[2]))

In [ ]:
print(test_outs.shape)
out = test_outs[0]
rows = round((out.shape[0] ** (1/2)))
columns = rows
print(rows, columns)
fig,axes = plt.subplots(rows,columns, figsize=(15,15))
for i in range(rows):
    for j in range(columns):
        if i*rows + j < out.shape[0]:
            axes[i][j].set_xticklabels([])
            axes[i][j].set_yticklabels([])
            axes[i][j].imshow(out[i*rows +j], cmap='gray')

In [ ]:
gap = np.mean(test_outs[0], axis=0)
plt.imshow(gap, cmap='gray')

In [ ]:
test_outs.shape

In [ ]:
def showFilters(layerName, reshape=False):
    blobs = alexnet.getParam(alexnet.getLayerId(layerName))
    s = blobs.shape
    if reshape:
        blobs = blobs.reshape(s[0], s[2], s[3], s[1])
    
    r = round(s[0] ** (1/2))
    c = r
    fig,axes = plt.subplots(r,c, figsize=(10,10))
    
    for i in range(r):
        for j in range(c):
            if i*r + j < s[0]:
                axes[i][j].set_xticklabels([])
                axes[i][j].set_yticklabels([])
                x = blobs[i*r + j]
                x = x.reshape((x.shape[2], x.shape[0], x.shape[1]))
                x[0] += abs(x[0].min())
                x[1] += abs(x[1].min())
                x[2] += abs(x[2].min())
                x = x.reshape((x.shape[1], x.shape[2], x.shape[0]))
                axes[i][j].imshow(x)
    

In [ ]:
def showFiltersGray(layerName):
    blobs = alexnet.getParam(alexnet.getLayerId(layerName))
    s = blobs.shape
    r = 16
    c = 16
    fig,axes = plt.subplots(r,c, figsize=(10,10))
    
    for i in range(r):
        for j in range(c):
            # if i*r + j < s[0]:
                axes[i][j].set_xticklabels([])
                axes[i][j].set_yticklabels([])
                x = blobs[i][j]
                x += abs(x.min())
                # x = blobs[i*r + j]
                # x = x.reshape((x.shape[2], x.shape[0], x.shape[1]))
                # x[0] += abs(x[0].min())
                # x[1] += abs(x[1].min())
                # x[2] += abs(x[2].min())
                # x = x.reshape((x.shape[1], x.shape[2], x.shape[0]))
                axes[i][j].imshow(x, cmap='gray')
    

In [ ]:
showFilters('conv1', True)

In [ ]:
showFiltersGray('conv3')

In [ ]:
t = alexnet.getParam(alexnet.getLayerId('conv2'))
s = t.shape
t = t.reshape(s[0],s[2],s[3],s[1])

In [ ]:
s

In [ ]:
def pca(vecs):
    centered_data = vecs - np.mean(vecs)
    U, S, V = np.linalg.svd(centered_data, full_matrices=False)
    components = V
    coefficients = np.dot(U, np.diag(S))
    return coefficients

In [ ]:
images = [
    'amur/plain_reid_train/train/002086.jpg',
    'amur/plain_reid_train/train/001888.jpg',
    'amur/plain_reid_train/train/004457.jpg'
]

vecs = []
fig,axes = plt.subplots(len(images))
x = 0
for imgf in images:
    image = cv2.imread(imgf)
    # test_img = get_bounding_boxes(image)
    test_img = image
    axes[x].imshow(test_img)
    x += 1
    feat = extract_feature(test_img, 'pool5')
    feat = getLogMFeature(feat)
    vecs.append(feat.reshape(np.prod(feat.shape)))
    # vecs.append(feat)

coeff = vecs # pca(vecs)
for i in range(len(coeff)):
    for j in range(i+1, len(coeff)):
        print('Similarity between Image {} and Image {} is {}'.format(i, j, cos_sim(coeff[i], coeff[j])))
    print()


In [ ]:
alexnet.

In [ ]:
def getLogMFeature(feat):
    temp = feat[:]
    temp = temp.reshape(np.prod(temp.shape))
    for i in range(len(temp)):
        if temp[i] == 0.0:
            temp[i] = 1e-8
    temp = temp.reshape(feat.shape)
    return np.tril(temp)

In [ ]:
test = vecs[0].reshape((1,384,13,13))

In [ ]:
fig,axes = plt.subplots(19,19, figsize=(15,15))
for i in range(19):
    for j in range(19):
        axes[i][j].set_xticklabels([])
        axes[i][j].set_yticklabels([])
        axes[i][j].imshow(test[0][i*19 + j], cmap='gray')

In [ ]:
gap = np.mean(test[0], axis=0,keepdims=True)

In [ ]:
r = tuple([x for x in range(13)])

In [ ]:
vecs[0].shape

In [ ]:
gap.re

In [ ]:
cos_sim(tiger, el_1)

In [ ]:
print(test_outs.shape)
out = test_outs[0]
rows = round((out.shape[0] ** (1/2)))
columns = rows
print(rows, columns)
fig,axes = plt.subplots(rows,columns, figsize=(15,15))
for i in range(rows):
    for j in range(columns):
        if i*rows + j < out.shape[0]:
            axes[i][j].set_xticklabels([])
            axes[i][j].set_yticklabels([])
            axes[i][j].imshow(out[i*rows +j], cmap='gray')

In [ ]:
labels[np.argmax(test_outs)]

In [ ]:
labels = []
with open('synset_words.txt') as f:
    labels = np.loadtxt(f, str, delimiter='\t')
    f.close()

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.threshold(gray, 0, 1, cv2.THRESH_OTSU)[1], cmap='gray')